In [210]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [211]:
import pandas as pd 
import numpy as np
!pip install openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import os
%matplotlib inline
from scipy import stats

In [212]:
df0 = pd.read_excel('../input/datainput-ne/data-input (2).xlsx')

In [213]:
df0.describe()

In [214]:
df0.drop(["Emiten","Net Profit Margin"], axis=1, inplace=True)

In [215]:
df0.info()

In [216]:
plt.figure(figsize=(15, 15))
sns.pairplot(df0, vars = df0.columns[1:13]);

In [217]:
df0 = pd.read_excel('../input/datainput-ne/data-input (2).xlsx')
df1 = df0[['DER','Return Of Asset','Sales Growth','Category']]
df1.describe()

In [218]:
plt.figure(figsize=(18, 4))
plt.xticks(rotation=45)
sns.scatterplot(x='Emiten', y='Return Of Asset',
                data=df0, s=80)

In [219]:
plt.figure(figsize=(18, 4))
plt.xticks(rotation=45)
sns.scatterplot(x='Emiten', y='DER', 
                data=df0, s=80)

In [220]:
plt.figure(figsize=(18, 4))
plt.xticks(rotation=45)
sns.scatterplot(x='Emiten', y='Sales Growth', 
                data=df0, s=80)

In [221]:
print(f'Jumlah baris sebelum memfilter outlier: {len(df1)}')

filtered_entries = np.array([False] * len(df1))
for col in ['Return Of Asset']:
    zscore = abs(stats.zscore(df1[col]))
    filtered_entries = (zscore < 3) | filtered_entries
    
df1 = df1[filtered_entries]

print(f'Jumlah baris setelah memfilter outlier: {len(df1)}')

In [222]:
df1.describe()

In [223]:
feats = ['DER','Return Of Asset','Sales Growth']
X = df1[feats].values
y = df1['Category'].values

from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)
new_df = pd.DataFrame(data = X_std, columns = feats)
new_df.describe()

In [224]:
from sklearn.cluster import KMeans
inertia = []

for i in range(1, 7):
  kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
  kmeans.fit(new_df.values)
  inertia.append(kmeans.inertia_)

plt.figure(figsize=(12, 6))
plt.plot(inertia) #Elbow Method

In [225]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, init='k-means++', max_iter=300, n_init=10, random_state=0)
kmeans.fit(new_df.values)

In [226]:
new_df['target'] = y
new_df['fit'] = kmeans.labels_

In [227]:
plt.rcParams["figure.figsize"] = (8, 8)
sns.scatterplot(x='DER', y='Return Of Asset', hue=new_df.target.to_list(), data=new_df);

In [228]:
new_df.head(5)

In [229]:
new_df.insert(0,"Category",np.nan,True)
new_df.loc[new_df['fit'] == 0, 'Category'] = 2
new_df.loc[new_df['fit'] == 1, 'Category'] = 4
new_df.loc[new_df['fit'] == 2, 'Category'] = 1
new_df.loc[new_df['fit'] == 3, 'Category'] = 0
new_df.loc[new_df['fit'] == 4, 'Category'] = 3
new_df['Category'] = new_df['Category'].astype(int)
new_df.head(5)

In [230]:
new_df.drop(["fit","target"], axis=1, inplace=True)
new_df.columns = ['Category','DER (St.)','Return Of Asset (St.)', 'Sales Growth (St.)']
new_df.info()

In [231]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')


x = new_df['DER (St.)']
y = new_df['Return Of Asset (St.)']
z = new_df['Sales Growth (St.)']

ax.set_xlabel("DER (St.)")
ax.set_ylabel("R Of Asset (St.)")
ax.set_zlabel("Sales Growth (St.)")

cmap = ListedColormap(sns.color_palette("RdYlGn", 256).as_hex())

sc = ax.scatter(x, y, z, s=40, c=new_df['Category'], marker='o', cmap=cmap, alpha=1)
plt.legend(*sc.legend_elements(), bbox_to_anchor=(1.05, 1), loc=2)

plt.show()

In [232]:
# color palette as dictionary
palette = {0:"tab:red",
           1:"tab:orange", 
           2:"yellow",
           3:"lime",
           4:"darkgreen"}

sns.scatterplot(x='DER (St.)', y='Sales Growth (St.)', 
                hue='Category', data=new_df, s=80,
                palette=palette)
plt.title("DER vs Sales Growth (Standardisasi)", fontsize=18, y=1.05)
plt.show()

In [233]:
sns.scatterplot(x='Return Of Asset (St.)', y='Sales Growth (St.)', 
                hue='Category', data=new_df, s=80,
                palette=palette)
plt.title("Return Of Asset vs Sales Growth (Standardisasi)", fontsize=18, y=1.05)
plt.show()

In [234]:
sns.scatterplot(x='DER (St.)', y='Return Of Asset (St.)', 
                hue='Category', data=new_df, s=80,
                palette=palette)
plt.title("DER vs Return Of Asset (Standardisasi)", fontsize=18, y=1.05)
plt.show()

In [235]:
# drop these row indexes
# from dataFrame
index_names = df0[ df0['Emiten'] == 'ARTO' ].index
df0.drop(index_names, inplace = True)
df0.reset_index(drop=True, inplace=True)
df0.drop(["Category",
          "Net Profit Margin",
         ], axis=1, inplace=True)

In [236]:
dftemp = pd.read_excel('../input/industryemiten-perbankan/industry-emiten-list.xlsx')
df = pd.concat([dftemp, new_df, df0], axis=1)
df.head(5)

In [237]:
df.to_excel('../input/resultcompan/result (1).xlsx', index = False)